# Parse text

In [26]:
import re

In [7]:
fname = './amazon-meta.txt'
flist = []
ix = 0
ix_list = [-1]
with open(fname,'r+',encoding='utf-8') as f:
    for line in f.readlines():
        flist.append(line)
        if line=='\n': ix_list.append(ix)
        ix += 1

In [37]:
item_list = []
for i in range(len(ix_list)-1):
    ix_start = ix_list[i]+1
    ix_end = ix_list[i+1]
    item_list.append(flist[ix_start:ix_end])

In [102]:
pattern = '(\s*[Ii]d\:)|(\s*[Gg]roup\:)|(\s*[Cc]ategories\:)'
pattern_ID = '(\s*[Ii]d\:)'
pattern_GP = '(\s*[Gg]roup\:)'
pattern_CAT = '(\s*[Cc]ategories\:)'
pattern_NUM = '\d+'
pattern_GV = '(\s*[Gg]roup\:)(.*)'

In [69]:
item_list_cleaned = []
for item in item_list:
    to_append=[]
    for line in item:
        if re.match(pattern, line):
            to_append.append(line)
    item_list_cleaned.append(to_append)
item_list_cleaned = item_list_cleaned[2:]

In [168]:
result = []
for item in item_list_cleaned:
    to_append = {'ID':0,'Group':'','Categories':0}
    for line in item:
        if re.match(pattern_ID, line):
            to_append['ID'] = int(re.findall(pattern_NUM, line)[0])
        if re.match(pattern_GP, line):
            to_append['Group'] = re.match(pattern_GV, line).group(2).strip()
        if re.match(pattern_CAT, line):
            to_append['Categories'] = int(re.findall(pattern_NUM, line)[0])
    result.append(to_append)

In [323]:
result_df = pd.DataFrame(result)

# filter nodes with few links

In [492]:
from sklearn.model_selection import train_test_split

In [493]:
data = pd.read_csv('Amazon0302.csv')

In [529]:
result_df = pd.DataFrame(result)

In [530]:
# stratified sampling
counts = result_df['Categories'].value_counts()
result_df = result_df[result_df['Categories'].isin(counts[counts>1].index)]

result_df_X = result_df[['ID','Group']]
result_df_y = result_df[['Categories']]

X_train, X_test, y_train, y_test = train_test_split(result_df_X, result_df_y, test_size=0.05, stratify=result_df_y)

result_df = X_test.copy()
result_df['Categories'] = y_test
result_df = result_df[result_df['ID']!=0]
result_df = result_df[result_df['Group']!='']

In [496]:
data_out = data[(data['ToNodeId'].isin(result_df['ID']))&(data['FromNodeId'].isin(result_df['ID']))]

# reset item id
id_list_raw = list(set(data_out['ToNodeId'])|set(data_out['FromNodeId']))
id_list_raw = sorted(id_list_raw)

id_convert = {}
for i, id_ in enumerate(id_list_raw):
    id_convert[id_]=i+1
    
data_out['FromNodeId'] = data_out['FromNodeId'].map(id_convert)
data_out['ToNodeId'] = data_out['ToNodeId'].map(id_convert)

/Users/JoeCheung/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/JoeCheung/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [497]:
result_df['ID_NEW'] = result_df['ID'] 
result_df['ID_NEW'] = result_df['ID_NEW'].map(id_convert)
result_df = result_df[~result_df['ID_NEW'].isna()]
result_df = result_df.sort_values('ID_NEW')

In [498]:
data_out.to_csv('Amazon0302_SUBSET.csv', index=False)

In [507]:
result_df.to_csv('ID_INFO_SUBSET.csv', index=False)

In [508]:
result_df

,ID,Group,Categories,ID_NEW
5,6,Book,5,1.0
9,10,Book,3,2.0
19,20,Book,3,3.0
27,28,Book,2,4.0
28,29,Book,5,5.0
...,...,...,...,...
261670,261671,Book,2,3580.0
261863,261864,Video,10,3581.0
261963,261964,Book,3,3582.0
262085,262086,Book,3,3583.0


In [500]:
data_out

,FromNodeId,ToNodeId
34,1,3
95,3,1
134,4,3
270,8,7
960,12,4
...,...,...
1232925,3580,728
1233785,3581,3582
1234225,3582,3581
1234761,3583,3582


In [532]:
pd.DataFrame(result_df['Categories'].value_counts().sort_index()).rename(columns={'Categories':'items'})

,items
0,1149
1,2732
2,4829
3,4782
4,4122
5,2600
6,1957
7,1157
8,950
9,550


In [513]:
tmp = pd.DataFrame(result)

In [528]:
pd.DataFrame(tmp['Categories'].value_counts().sort_index()).rename(columns={'Categories':'items'})

,items
0,28770
1,54645
2,96585
3,95636
4,82447
...,...
90,1
95,1
97,1
101,2
